In [11]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

In [12]:
# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      

In [13]:
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

In [14]:
df[df['station_nbr'] == 1].describe()

,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,103119.000000,103119.000000,103119.000000,103119.0,103119.0,103119.000000,103119.000000,103119.00000,103119.000000
mean,0.493003,0.500538,0.079656,1.0,1.0,56.000000,0.364909,3.00000,0.285253
std,0.499953,0.500002,0.270761,0.0,0.0,32.041795,3.609916,2.00001,0.451537
min,0.000000,0.000000,0.000000,1.0,1.0,1.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,1.0,1.0,28.000000,0.000000,1.00000,0.000000
50%,0.000000,1.000000,0.000000,1.0,1.0,56.000000,0.000000,3.00000,0.000000
75%,1.000000,1.000000,0.000000,1.0,1.0,84.000000,0.000000,5.00000,1.000000
max,1.000000,1.000000,1.000000,1.0,1.0,111.000000,207.000000,6.00000,1.000000


In [4]:
# 위에까지는 다 똑같음

In [7]:
# station1 data = df1
df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [8]:
# df 모든 'M' and '-' 을 np.nan으로 형 변환 하는 과정
df1 = df1.apply(pd.to_numeric, errors = 'coerce')

In [9]:
# 모든 Data가 Nan 값인 Columns
df1_drop_columns = ['date', 'depart', 'sunrise', 'sunset', 'codesum', 'snowfall', 'store_nbr', 'station_nbr']
df1 = df1.drop(columns = df1_drop_columns)

ValueError: labels ['date' 'depart' 'sunrise' 'sunset' 'codesum' 'snowfall' 'store_nbr'
 'station_nbr'] not contained in axis

In [10]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df1_columns = df1.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df1_columns:
    if (i == 'resultdir'):
        df1[i].fillna(df1[i].mode()[0], inplace=True)
        print(df1[i].mode()[0])
    else:
        df1[i].fillna(df1[i].mean(), inplace=True)

21.0


In [11]:
# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df1['relative_humility'] = 100*(np.exp((17.625*((df1['dewpoint']-32)/1.8))/(243.04+((df1['dewpoint']-32)/1.8)))/np.exp((17.625*((df1['tavg']-32)/1.8))/(243.04+((df1['tavg']-32)/1.8))))

# 체감온도 계산
df1["windchill"] = 35.74 + 0.6215*df1["tavg"] - 35.75*(df1["avgspeed"]**0.16) + 0.4275*df1["tavg"]*(df1["avgspeed"]**0.16)

In [12]:
# Station1에 대한 전처리 완료 
df1.columns

Index(['tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'nothing', 'rain', 'other', 'item_nbr', 'units', 'week7',
       'weekend', 'relative_humility', 'windchill'],
      dtype='object')

In [13]:
# no units은 삭제
df1 = df1[df1['units'] != 0]

In [14]:
model_df1 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + wetbulb + heat + cool + preciptotal + stnpressure + \
sealevel + resultspeed + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + tmax:tmin + tmax:tavg + tmax:dewpoint + 0', data = df1)

result_df1 = model_df1.fit()

print(result_df1.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.621
Method:                 Least Squares   F-statistic:                     143.8
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        19:02:18   Log-Likelihood:                -2636.7
No. Observations:                2877   AIC:                             5341.
Df Residuals:                    2843   BIC:                             5544.
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(nothing)[0]         0.0831      0.07

In [15]:
anova_result_df1 = sm.stats.anova_lm(result_df1, typ=2).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df1

,sum_sq,df,F,PR(>F)
tmin,0.000307,1.0,0.000829,9.770285e-01
heat,0.003561,1.0,0.009612,9.219067e-01
resultspeed,0.005188,1.0,0.014005,9.058048e-01
tmax:tmin,0.021937,1.0,0.059219,8.077517e-01
preciptotal,0.041312,1.0,0.111523,7.384411e-01
resultdir,0.044151,1.0,0.119187,7.299436e-01
relative_humility,0.074688,1.0,0.201621,6.534496e-01
dewpoint,0.136280,1.0,0.367890,5.442053e-01
tmax:dewpoint,0.176249,1.0,0.475786,4.903935e-01
C(nothing),0.540048,2.0,0.728933,4.825135e-01


In [17]:
model_df1_1 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool) + scale(preciptotal) + scale(stnpressure) + \
scale(sealevel) + scale(resultspeed) + scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df1)

result_df1_1 = model_df1_1.fit()

print(result_df1_1.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.621
Method:                 Least Squares   F-statistic:                     158.0
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        19:02:34   Log-Likelihood:                -2638.6
No. Observations:                2877   AIC:                             5339.
Df Residuals:                    2846   BIC:                             5524.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [18]:
anova_result_df1_1 = sm.stats.anova_lm(result_df1_1, typ=2).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df1_1[anova_result_df1_1['PR(>F)'] <= 0.05]

,sum_sq,df,F,PR(>F)
scale(windchill),2.938584,1.0,7.930568,4.894133e-03
C(weekend),21.254623,1.0,57.361391,4.871454e-14
C(week7),63.262133,6.0,28.455017,3.295240e-33
C(item_nbr),1677.021258,7.0,646.556978,0.000000e+00
C(nothing),2550.848954,2.0,3442.080496,0.000000e+00


In [19]:
# 이외의 변수 없애고, 다시 회귀 분석
model_df1_2 = sm.OLS.from_formula('np.log(units) ~ scale(windchill) + C(weekend) \
+ C(item_nbr) + C(nothing) + 0', data = df1)

result_df1_2 = model_df1_2.fit()

print(result_df1_2.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(units)   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     435.5
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        19:02:56   Log-Likelihood:                -3196.0
No. Observations:                2877   AIC:                             6414.
Df Residuals:                    2866   BIC:                             6480.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(weekend)[0]         3.0108      0.02

In [20]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df1.values, i) for i in range(df1.shape[1])]
vif["features"] = df1.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif

,VIF Factor,features
0,1.255894,other
1,1.290353,item_nbr
2,1.317498,units
3,1.519798,resultdir
4,1.542944,preciptotal
5,2.717016,week7
6,2.750141,weekend
7,15.232990,resultspeed
8,22.927143,avgspeed
9,37.720286,rain


In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df1.drop(columns=['relative_humility', 'rain', 'nothing', 'wetbulb', 'dewpoint', 'tmin', 'avgspeed', 'tavg', 'stnpressure', 'sealevel', 'windchill', 'tmax', 'heat']).values, i) for i in range(df1.drop(columns=['relative_humility', 'rain', 'nothing', 'wetbulb', 'dewpoint', 'tmin', 'avgspeed', 'tavg', 'stnpressure', 'sealevel', 'windchill', 'tmax', 'heat']).shape[1])]
vif["features"] = df1.drop(columns=['relative_humility', 'rain', 'nothing', 'wetbulb', 'dewpoint', 'tmin', 'avgspeed', 'tavg', 'stnpressure', 'sealevel', 'windchill', 'tmax', 'heat']).columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif

,VIF Factor,features
0,1.111400,other
1,1.189486,preciptotal
2,1.220802,cool
3,1.764599,units
4,2.577880,item_nbr
5,3.416128,resultspeed
6,3.567587,weekend
7,4.264403,resultdir
8,7.369288,week7


In [22]:
# 이외의 변수 없애고, 다시 회귀 분석
model_df1_3 = sm.OLS.from_formula('np.log1p(units) ~ C(weekend)  \
+ C(item_nbr) + 0', data = df1)

result_df1_3 = model_df1_3.fit()

print(result_df1_3.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.616
Method:                 Least Squares   F-statistic:                     576.7
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        19:03:33   Log-Likelihood:                -2669.5
No. Observations:                2877   AIC:                             5357.
Df Residuals:                    2868   BIC:                             5411.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(weekend)[0]         3.0834      0.02

In [28]:
## K-fold Cross Validation
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X1 = df1[['weekend', 'item_nbr']]
y1 = df1['units']

model = LinearRegression()



cv1 = KFold(n_splits=10, shuffle=True, random_state=0)

for i, (train_index, test_index) in enumerate(cv1.split(X1)):
    X1_train = X1.values[train_index]
    y1_train = y1.values[train_index]
    X1_test = X1.values[test_index]
    y1_test = y1.values[test_index]
    model.fit(X1_train, y1_train)
    y1_pred = model.predict(X1_test)
    scores[i] = r2_score(y1_test, y1_pred)

scores

TypeError: 'str' object does not support item assignment

In [29]:
from sklearn.model_selection import cross_val_score

cross_val_score(model, X1, y1, scoring="r2", cv=cv1)

array([0.23952683, 0.19059646, 0.22654294, 0.28623825, 0.22997164,
       0.20538701, 0.22822145, 0.23097587, 0.23404436, 0.21127868])

In [30]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X1 = df1[['week7', 'weekend', 'item_nbr']]
Y1 = df1['units']

model = LinearRegression()


cv1 = KFold(n_splits=10, shuffle=True, random_state=0)

for i, (train_index, test_index) in enumerate(cv1.split(X1)):
    X1_train = X1.values[train_index]
    y1_train = Y1.values[train_index]
    X1_test = X1.values[test_index]
    y1_test = Y1.values[test_index]
    model.fit(X1_train, y1_train)
    y1_pred = model.predict(X1_test)
    scores[i] = r2_score(y1_test, y1_pred)

scores

TypeError: 'str' object does not support item assignment

In [ ]:
X_train, X_train.shape

In [ ]:
y_train, y_train.shape

In [31]:
X, y, coef = make_regression(n_samples=1000, n_features=3, noise=20, coef=True, random_state=0)

model = LinearRegression()
cv = KFold(10)

scores = np.zeros(10)
for i, (train_index, test_index) in enumerate(cv.split(X)):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores[i] = r2_score(y_test, y_pred)

scores

array([0.9821393 , 0.97851676, 0.96995914, 0.97776639, 0.96941406,
       0.96876349, 0.97595203, 0.97213353, 0.98444921, 0.97364191])

In [ ]:
# ls = []
# for i in df1.columns:
#     if (i == 'nothing') | (i == 'rain') | (i == 'other') | (i == 'item_nbr') | (i == 'week7') | (i == 'weekend'):
#         for j in df1.columns:
#             if (j == 'nothing') | (j == 'rain') | (j == 'other') | (j == 'item_nbr') | (j == 'week7') | (j == 'weekend'): 
#                 ls.append("C({}):C({})".format(i, j))
#             else:
#                 ls.append('C({}):{}'.format(i,j))
#     else:
#         for j in df1.columns:
#             if (j == 'nothing') | (j == 'rain') | (j == 'other') | (j == 'item_nbr') | (j == 'week7') | (j == 'weekend'): 
#                 ls.append("{}:C({})".format(i, j))
#             else:
#                 ls.append('{}:{}'.format(i,j))
